# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [8]:
from selenium import webdriver
# #importampos libreria para cargar el driver automaticamente
# from webdriver_manager.firefox import GeckoDriverManager
#importampos libreria para cargar el driver manualemente
from selenium.webdriver.chrome.options import Options
import time
#pagina web que vamos a hacer el scraping
url='https://www.latamairlines.com/ar/es/ofertas-vuelos?origin=ASU&inbound=null&outbound=2022-12-01T15%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

## Demora Dinamica
<p>Para evitar fallos de carga de la página y de mensajes emergente que no dejen realizar el scraper, se implementan demoras dinámicas, estas a diferencia de la demora estática brindan un mejor rendimiento. Ya que no tienen que esperar el tiempo estático sino que esperan a que se cargue la página y si tarda menos del tiempo estipulado, empieza a hacer el scraper.</p>

 Lo mejor sería esperar a que la página termine de cargar y luego recuperar los elementos. para ello importamos varias librerias

In [9]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [10]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    time.sleep(5)
    tarifas= vuelo.find_elements('xpath','.//ol[@class="sc-iAVDmT DwgCo"]/li[@class="sc-jjgyjb buqrVI"]')
    precios=[]
    for tarifa in tarifas:
        #buscamos en cada pocision de la tarifa los siquientes elementos
        nombre = tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[1]/span[@class="sc-gBSKhj chxCgG"]').text
        moneda= tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[3]//span[contains(@class,"currency")]').text
        valor= tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[3]//span[@class="sc-ihiiSJ AoaXI"]').text
        #guardo los valores que obtengo en un diccionario
        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
        #guradamos nuestro diccionario con los datos de tarifa a nuestra lia de precio
        precios.append(dict_tarifa)
    return precios

In [11]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de 
    las escalas de cada vuelo
    '''

    time.sleep(5)
    print("Itinerario de vuelo termino de Cargar")

    segmentos = vuelo.find_elements('xpath', '//div[contains(@class,"MuiDialogContent-root")]/article//div/section')
    # segmentos= vuelo.find_elements('xpath','.//div[@class="MuiDialogContent-root sc-fjdhpX jvVjjf"]//section')

    datos_escalas=[]    
    for segmento in segmentos: 
        time.sleep(10)  
        #origen
        origen=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[1]').text
        #hora de salida
        hora_salida=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[last()]').text
        #destino
        destino_segmento=segmento.find_element('xpath','.//div[@class="sc-hAcydR ikJhgn"]/div[@class="iataCode"]/span[1]').text
        #Hora de llegada
        hora_llegada=segmento.find_element('xpath','.//div[@class="sc-hAcydR ikJhgn"]/div[@class="iataCode"]/span[2]').text
        #Duracion del vuelo
        duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-BOulX kmIWrd"]/span[2]').text
        #Numero de vuelo
        numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//div[@class="sc-bscRGj iggkUa airline-wrapper"]').text
        #Modelo de avion
        modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//span[@class="airplane-code"]').text
        
        #el último segmento no tendrá escala por que es el destino final del vuelo
        if segmento != segmentos[-1]:    
            #Duracion escala
            duracion_escala_vuelo=segmento.find_element('xpath','.//div[@class="sc-fdJbru eZIckp"]//span[@class="time"]').text
        else:
            duracion_escala_vuelo=''
        
        # Armo un diccionario para almacenar los datos
        datos_escalas_dict={
            'origen':origen,
            'hora_salida':hora_salida,
            'destino':destino_segmento,
            'hora_llega':hora_llegada,
            'duracion':duracion_segmento,
            'numero_vuelo':numero_vuelo_segmento,
            'modelo_avion':modelo_avion_segmento,
            'duracion_escala':duracion_escala_vuelo}
        datos_escalas.append(datos_escalas_dict)
        print (f'{datos_escalas}')
    return datos_escalas

In [12]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    #hora de salida
    hora_salida=vuelo.find_element('xpath','.//div[@class="sc-ixltIz dfdfxH flight-information"]/span[1]').text
    #hora de llegada
    hora_llegada=vuelo.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
    # Duracion del vuelo
    duracion_vuelo= vuelo.find_element('xpath','.//div[2]/span[2]').text

    tiempos_vuelo_dict={
        'hora_salida':hora_salida,
        'hora_llegada':hora_llegada,
        'duracion_vuelo':duracion_vuelo
    }

    return tiempos_vuelo_dict

In [13]:
def obtener_info(driver):
    #Usaremos el Xpath para obtener la lista de vuelos
    vuelos = driver.find_elements('xpath','//ol/li')
    print(f'Se encontaron {len(vuelos)} vuelos.')
    print(f'Iniciando Scraping...')
    info=[]
    contador=0
    for vuelo in vuelos:
        if contador<3:
            contador+=1
            print(f'Analizando {contador} de {len(vuelos)} vuelos')

            #obtener los tiempos generales de cada vuelo
            tiempos=obtener_tiempos(vuelo)
            print(f'{tiempos}\n')
            
            #clickeamos sobe el link escalas para que se habra el modal
            vuelo.find_element('xpath','//a[contains(@id,"itinerary-modal")]').click()

            #obtener escalas
            escalas=obtener_datos_escalas(vuelo)
            # print(f'{escalas}\n')
            
            #cerramos el modal
            driver.find_element('xpath','//*[@id="itinerary-modal-0-dialog-close"]').click()

            #clickeamos el vuelo para ver los precios
            vuelo.click()

            print('La tabla de precios terminó de cargar')
            precios=obtener_precios(vuelo)
            print(f'{precios}\n*********************************************\n')
            
            #click nuevamente para cerrar los precios que se desplegaron
            vuelo.find_element('xpath','//div[@id="undefined--button-wrapper"]//button').click()

            info.append({'precios':precios,'tiempos':tiempos,'escalas':escalas})

    return info

In [14]:
#cargar drive automaticamente utilizando webdriver
# options = webdriver.FirefoxOptions()
#carga driver manualmente
options= Options()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('--incognito')
#cargar drive automaticamente utilizando webdriver
#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)
#carga driver manualmente
driver = webdriver.Chrome(options=options)
driver.maximize_window()

driver.get(url)
def close_dialog():
    '''
    Función que cierra los dialog o popups de la página
    '''
    dialogs = driver.find_elements(By.XPATH, '//div[contains(@class,"MuiDialog-container")]')
    if len(dialogs):
        dialogs[0].find_element(By.XPATH, './/button[contains(@class,"Dialog__CloseButton")]').click()
close_dialog()

try:
    # introducir demora inteligente
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="WrapperBodyFlights"]/ol/li')))
    print('La página terminó de cargar')
    obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
finally:
    print('error')
    #driver.close()

La página terminó de cargar
Se encontaron 30 vuelos.
Iniciando Scraping...
Analizando 1 de 30 vuelos
{'hora_salida': '10:50', 'hora_llegada': '13:55', 'duracion_vuelo': '23 h 5 min'}

Itinerario de vuelo termino de Cargar
error


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="sc-fdJbru eZIckp"]//span[@class="time"]"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x0041ACD3+2075859]
	Ordinal0 [0x003AEE61+1633889]
	Ordinal0 [0x002AB7BD+571325]
	Ordinal0 [0x002DAC2F+764975]
	Ordinal0 [0x002DAE1B+765467]
	Ordinal0 [0x002D1681+726657]
	Ordinal0 [0x002F7364+881508]
	Ordinal0 [0x002D15BF+726463]
	Ordinal0 [0x002F7534+881972]
	Ordinal0 [0x0030B56A+963946]
	Ordinal0 [0x002F7136+880950]
	Ordinal0 [0x002CFEFD+720637]
	Ordinal0 [0x002D0F3F+724799]
	GetHandleVerifier [0x006CEED2+2769538]
	GetHandleVerifier [0x006C0D95+2711877]
	GetHandleVerifier [0x004AA03A+521194]
	GetHandleVerifier [0x004A8DA0+516432]
	Ordinal0 [0x003B682C+1665068]
	Ordinal0 [0x003BB128+1683752]
	Ordinal0 [0x003BB215+1683989]
	Ordinal0 [0x003C6484+1729668]
	BaseThreadInitThunk [0x76CDFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77B07BBE+286]
	RtlGetAppC

Paso 4: cerrar el navegador

In [ ]:
driver.close()